# Enable APIs (Container, Kubernetes) in Google Cloud Platform

# Create cluster in console

# Install Google Cloud SDK

# Configure Docker to use gcloud tool to authenticate requests
> gcloud auth configure-docker

# Download auth token key from console and put it in the directory
> API & Services -> Credentials: create credentials > Service account key

# Fetch cluster endpoint and auth data and update kubeconfig entry

> gcloud container clusters get-credentials mm-docker-models-gke --zone us-east4-a [--project <project name>]

# Configure firewall to allow the node port range (30000-32767)
> gcloud compute firewall-rules create gke-rule --allow tcp:30000-32767


# Configure 'GCP' section in cli.properties


In [1]:
from mm_docker_lib import *
initConfig("GCP")

Loading configuration properties...
Login into GCP GCR...
Login GCP GCR succeeded!
Initializing kubernetes configuration... gke_modelmanager_us-east4-a_mm-docker-models-gke
Initialized kubernetes configuration.
  verbose: False
  model.repo.host: http://honxin.modelmanager.sashq-d.openstack.sas.com
  provider.type: GCP
  base.repo: gcr.io/modelmanager/
  base.repo.web.url: console.cloud.google.com/gcr/images/modelmanager/GLOBAL/
  kubernetes.context: gke_modelmanager_us-east4-a_mm-docker-models-gke


True

In [3]:
listmodel("svm")

Model name svm (pipeline 1)
Model UUID d00bb4e3-0672-4e9a-a877-39249d2a98ab
Model version 63.0
Project name MySVM
Score Code Type ds2MultiType
Image URL (not verified): gcr.io/modelmanager/svm-pipeline-1_d00bb4e3-0672-4e9a-a877-39249d2a98ab:latest


In [4]:
publish("d00bb4e3-0672-4e9a-a877-39249d2a98ab")

Copying astore from shared directory...
Building image...
Pushing to repo...
Pushed. Please verify it at console.cloud.google.com/gcr/images/modelmanager/GLOBAL/svm-pipeline-1_d00bb4e3-0672-4e9a-a877-39249d2a98ab/
Model image URL: gcr.io/modelmanager/svm-pipeline-1_d00bb4e3-0672-4e9a-a877-39249d2a98ab:latest


'gcr.io/modelmanager/svm-pipeline-1_d00bb4e3-0672-4e9a-a877-39249d2a98ab:latest'

In [5]:
launch("gcr.io/modelmanager/svm-pipeline-1_d00bb4e3-0672-4e9a-a877-39249d2a98ab:latest")

Validating image repository url...
Completed validation.
Launching container instance...
Deployment created. 
Deployment name:  svm-pipeline-1-cb6kcd
Service created.
Getting service url...
Service URL:  http://35.194.76.110:31415
Checking whether the instance is up or not...
1 ==Sleep 10 seconds...
Checking whether the instance is up or not...
Instance is up!


('svm-pipeline-1-cb6kcd', 'http://35.194.76.110:31415')

In [6]:
execute("http://35.194.76.110:31415", "hmeq.csv")

Performing scoring in the container instance...
The test_id from score execution: 1551893540.5150483


'1551893540.5150483'

In [7]:
query(service_url="http://35.194.76.110:31415",test_id="1551893540.5150483")

The test result has been retrieved and written into file 1551893540.5150483.csv
Head is the first 5 lines
EM_CLASSIFICATION,EM_EVENTPROBABILITY,EM_PROBABILITY,I_BAD,P_BAD0,P_BAD1,_P_,_WARN_
0                               ,3.5255933e-05,0.9999648,0                               ,0.9999648,3.5255933e-05,1.0000224,
0                               ,4.455951e-05,0.9999554,0                               ,0.9999554,4.455951e-05,1.0000038,
0                               ,3.431873e-05,0.99996567,0                               ,0.99996567,3.431873e-05,1.0000242,
1                               ,1.0,1.0,1                               ,0.0,1.0,-1.1583366,


'1551893540.5150483.csv'

In [8]:
stop(deployment_name="svm-pipeline-1-cb6kcd")

Deleting service svm-pipeline-1-cb6kcd
deleted svc/svm-pipeline-1-cb6kcd from ns/default
Deleting app deployment... svm-pipeline-1-cb6kcd
Deletion succeeded


In [ ]:
score("docker.sas.com/honxin/svm-pipeline-1_d00bb4e3-0672-4e9a-a877-39249d2a98ab:latest", "hmeq.csv")

=====================

In [ ]:
listmodel("xgboost")

In [ ]:
setVerbose(True)
publish("2fd01d3e-ac53-406d-86cd-ac3cc9557c57")

In [ ]:
setVerbose(True)
launch("docker.sas.com/honxin/xgboost_2fd01d3e-ac53-406d-86cd-ac3cc9557c57:latest")

In [ ]:
execute("http://10.23.13.194:30456","xg_sample.csv")

In [ ]:
help(query)

In [ ]:
pwd